## Extraccion, Tranformacion y Carga

In [1]:
import json             # importo modulo json encoder and decoder
import pandas as pd     # importo modulo panda Data Analysis Library

Una vez importados los modulos, importo el dataset e inicio mi trabajo con pandas

      Incorporo el archivo output_steam_games

In [2]:
rows = []
with open ('Data/output_steam_games.json', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        rows.append(json.loads(line))

# save to dF
df_steam_games = pd.DataFrame(rows)


Muestro el dataframe

In [3]:
df_steam_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


Se observan filas que contienen vacíos en todo el registro. Se eliminan dichos registros.

In [4]:
df_steam_games = df_steam_games.dropna(how='all').reset_index(drop=True)
df_steam_games.shape

(32135, 13)

Se revisa los tipos de datos de cada columna, a su vez se verifica la existencia de nulos en cada una de ellas.

In [5]:
def obtener_informacion_columnas(df):
    mi_dict = {"nombre_campo": [], "tipo_datos": [], "no_nulos_%": [], "nulos_%": [], "nulos": []}

    for columna in df.columns:
        porcentaje_no_nulos = (df[columna].count() / len(df)) * 100
        mi_dict["nombre_campo"].append(columna)
        mi_dict["tipo_datos"].append(df[columna].dtype)
        mi_dict["no_nulos_%"].append(round(porcentaje_no_nulos, 2))
        mi_dict["nulos_%"].append(round(100 - porcentaje_no_nulos, 2))
        mi_dict["nulos"].append(df[columna].isnull().sum())

    df_info = pd.DataFrame(mi_dict)
    return df_info
informacion_columnas = obtener_informacion_columnas(df_steam_games)
print(informacion_columnas)

    nombre_campo tipo_datos  no_nulos_%  nulos_%  nulos
0      publisher     object       74.94    25.06   8052
1         genres     object       89.78    10.22   3283
2       app_name     object       99.99     0.01      2
3          title     object       93.62     6.38   2050
4            url     object      100.00     0.00      0
5   release_date     object       93.57     6.43   2067
6           tags     object       99.49     0.51    163
7    reviews_url     object       99.99     0.01      2
8          specs     object       97.92     2.08    670
9          price     object       95.71     4.29   1377
10  early_access     object      100.00     0.00      0
11            id     object       99.99     0.01      2
12     developer     object       89.73    10.27   3299


Se verifica el contenido de la columna 'Genres'

In [6]:
print(df_steam_games['genres'])

0            [Action, Casual, Indie, Simulation, Strategy]
1                     [Free to Play, Indie, RPG, Strategy]
2        [Casual, Free to Play, Indie, Simulation, Sports]
3                              [Action, Adventure, Casual]
4                                                      NaN
                               ...                        
32130                [Casual, Indie, Simulation, Strategy]
32131                            [Casual, Indie, Strategy]
32132                          [Indie, Racing, Simulation]
32133                                      [Casual, Indie]
32134                                                  NaN
Name: genres, Length: 32135, dtype: object


La columna 'Genres' tiene una lista con mas de un diccionario. Dentro de los diccionarios se distinguen las siguientes claves: 

    Info , check Json keys
       ['Action',
        'Casual',
         'Indie',
        'Simulation',
        'Strategy']  

La columna 'Genres' esta formada por una lista de los distintos géneros de los videojuegos; se encuentra anidada, por lo que se normaliza esta columna para obtener una columna con cada género.

In [7]:
dfreviewsOpen = df_steam_games.explode('genres')
df_steam_games = dfreviewsOpen.dropna(subset=['genres'])
df_steam_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
0,Kotoshiro,Casual,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
0,Kotoshiro,Indie,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
0,Kotoshiro,Simulation,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
0,Kotoshiro,Strategy,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32132,Laush Studio,Indie,Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
32132,Laush Studio,Racing,Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
32132,Laush Studio,Simulation,Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
32133,SIXNAILS,Casual,EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


Las columnas 'Tags', 'Specs', 'Url', 'Reviews_url', no se utilizarán para las consultas de la API ni para el modelo de recomendación, por lo tanto se eliminan del dataframe.

In [8]:
df_steam_games = df_steam_games.drop(['tags', 'reviews_url', 'specs', 'url'], axis=1)


In [9]:
df_steam_games

,publisher,genres,app_name,title,release_date,price,early_access,id,developer
0,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,4.99,False,761140,Kotoshiro
0,Kotoshiro,Casual,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,4.99,False,761140,Kotoshiro
0,Kotoshiro,Indie,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,4.99,False,761140,Kotoshiro
0,Kotoshiro,Simulation,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,4.99,False,761140,Kotoshiro
0,Kotoshiro,Strategy,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,4.99,False,761140,Kotoshiro
...,...,...,...,...,...,...,...,...,...
32132,Laush Studio,Indie,Russian Roads,Russian Roads,2018-01-04,1.99,False,610660,Laush Dmitriy Sergeevich
32132,Laush Studio,Racing,Russian Roads,Russian Roads,2018-01-04,1.99,False,610660,Laush Dmitriy Sergeevich
32132,Laush Studio,Simulation,Russian Roads,Russian Roads,2018-01-04,1.99,False,610660,Laush Dmitriy Sergeevich
32133,SIXNAILS,Casual,EXIT 2 - Directions,EXIT 2 - Directions,2017-09-02,4.99,False,658870,"xropi,stev3ns"


In [10]:
conteo_null = df_steam_games['id'].isnull().sum()  
print("Valores Nulos en la columna 'id':", conteo_null)

Valores Nulos en la columna 'id': 2


Se encontraron 2 valores NaN, por lo que se busca por la columna 'Developer' si este juego está ya registrado y si hay valores coincidentes en los registros.

In [11]:
df_steam_games[df_steam_games['developer']=='Rocksteady Studios,Feral Interactive (Mac)']

,publisher,genres,app_name,title,release_date,price,early_access,id,developer
1068,"Warner Bros. Interactive Entertainment, Feral ...",Action,Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,2012-09-07,19.99,False,200260,"Rocksteady Studios,Feral Interactive (Mac)"
1068,"Warner Bros. Interactive Entertainment, Feral ...",Adventure,Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,2012-09-07,19.99,False,200260,"Rocksteady Studios,Feral Interactive (Mac)"
30961,"Warner Bros. Interactive Entertainment, Feral ...",Action,Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,2012-09-07,19.99,False,NaN,"Rocksteady Studios,Feral Interactive (Mac)"
30961,"Warner Bros. Interactive Entertainment, Feral ...",Adventure,Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,2012-09-07,19.99,False,NaN,"Rocksteady Studios,Feral Interactive (Mac)"
31617,"Warner Bros. Interactive Entertainment, Feral ...",Action,Batman: Arkham Asylum Game of the Year Edition,Batman: Arkham Asylum Game of the Year Edition,2010-03-26,19.99,False,35140,"Rocksteady Studios,Feral Interactive (Mac)"
31617,"Warner Bros. Interactive Entertainment, Feral ...",Adventure,Batman: Arkham Asylum Game of the Year Edition,Batman: Arkham Asylum Game of the Year Edition,2010-03-26,19.99,False,35140,"Rocksteady Studios,Feral Interactive (Mac)"


Efectivamente, se puede ver que los items que no tienen 'id' es el 'id' 200260 y que el registro se encuentra completo. Por lo que se pueden borrar ambas filas.

In [12]:
# se eliminan los index deseados
index_a_eliminar = [30961]
df_steam_games = df_steam_games.drop(index_a_eliminar)
# se verifica si hay valores nulos
conteo_null = df_steam_games['id'].isnull().sum()  
print("Valores Nulos en la columna 'id':", conteo_null)

Valores Nulos en la columna 'id': 0


Verifico si existen duplicados

In [13]:
df_duplicate = df_steam_games.loc[df_steam_games.duplicated()]
df_duplicate

,publisher,genres,app_name,title,release_date,price,early_access,id,developer
14573,Bethesda Softworks,Action,Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,2017-10-26,59.99,False,612880,Machine Games


Se observa que solo hay un item duplicado. El 'id' 612880 que se observa es el mismo registro en los dos casos. Por lo que se puede eliminar cualquier de ellos. 

In [14]:
df_steam_games.loc[df_steam_games['id'] == '612880']

,publisher,genres,app_name,title,release_date,price,early_access,id,developer
13894,Bethesda Softworks,Action,Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,2017-10-26,59.99,False,612880,Machine Games
14573,Bethesda Softworks,Action,Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,2017-10-26,59.99,False,612880,Machine Games


In [15]:
df_steam_games = df_steam_games.drop_duplicates(keep='first')

Se extrae el año de la columna'Release date'

In [16]:
df_steam_games['release_date'] = df_steam_games['release_date'].str.extract(r'(\d{4})')

Verifico si en la columna 'Release_date' existen vlores nulos

In [17]:
conteo_null = df_steam_games['release_date'].isnull().sum()  
print("Valores Nulos en la columna 'release_date':", conteo_null)

Valores Nulos en la columna 'release_date': 338


Se detectaron 338 valores nulos, los cuales son reemplazados por el valor 'Sin dato'

In [18]:
# Reemplazar los valores nulos por 'Sin dato'
df_steam_games['release_date'].fillna('Sin dato', inplace=True)


Se necesita trabajar con la columna 'Price', pero hay valores de texto en casos de promociones o que el item es gratis. Por lo que se decide reemplazar esos valores por 0. Por otra parte, esta columna tiene algunos valores nulos, pero como mas adelante será necesario hacer operaciones, se adopta imputar como 0 los valores nulos.

In [19]:
def cambio_flotante(value):
    if pd.isna(value):
        return 0.0
    try:
        float_value = float(value)
        return float_value
    except:
        return 0.0
    
df_steam_games['price'] = df_steam_games['price'].apply(cambio_flotante)
df_steam_games['price'].dtype

dtype('float64')

En las columnas 'Publisher', 'App_name', 'Title', 'Developer' se observan valores nulos, por lo que se decide completarlos con valores del tipo 'Dato no disponible' para que el tipo de dato sea uno solo en la columna.

In [20]:
completar = ['publisher', 'app_name', 'title', 'developer']
# Se rellenan los nulos
df_completar = df_steam_games[completar].fillna('Dato no disponible')
# Se borran las columnas originales y se concatenan las rellenas con todo el dataframe
df_steam_games = pd.concat([df_steam_games.drop(completar, axis=1), df_completar], axis=1)
df_steam_games.head()

,genres,release_date,price,early_access,id,publisher,app_name,title,developer
0,Action,2018,4.99,False,761140,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
0,Casual,2018,4.99,False,761140,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
0,Indie,2018,4.99,False,761140,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
0,Simulation,2018,4.99,False,761140,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
0,Strategy,2018,4.99,False,761140,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro


Se verifica nuevamente los tipos de datos y los nulos que quedaron finalmente en el dataframe.

In [21]:
def obtener_informacion_columnas(df):
    mi_dict = {"nombre_campo": [], "tipo_datos": [], "no_nulos_%": [], "nulos_%": [], "nulos": []}

    for columna in df.columns:
        porcentaje_no_nulos = (df[columna].count() / len(df)) * 100
        mi_dict["nombre_campo"].append(columna)
        mi_dict["tipo_datos"].append(df[columna].dtype)
        mi_dict["no_nulos_%"].append(round(porcentaje_no_nulos, 2))
        mi_dict["nulos_%"].append(round(100 - porcentaje_no_nulos, 2))
        mi_dict["nulos"].append(df[columna].isnull().sum())

    df_info = pd.DataFrame(mi_dict)
    return df_info
informacion_columnas = obtener_informacion_columnas(df_steam_games)
print(informacion_columnas)

   nombre_campo tipo_datos  no_nulos_%  nulos_%  nulos
0        genres     object       100.0      0.0      0
1  release_date     object       100.0      0.0      0
2         price    float64       100.0      0.0      0
3  early_access     object       100.0      0.0      0
4            id     object       100.0      0.0      0
5     publisher     object       100.0      0.0      0
6      app_name     object       100.0      0.0      0
7         title     object       100.0      0.0      0
8     developer     object       100.0      0.0      0


## Carga de datos

Una vez realizada la limpieza de datos, se cargan los mismos en un nuevo archivo `df_steam_games.csv`

In [22]:
df_steam_games.to_csv('Datos/df_steam_games.csv', index=False)